In [1]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

In [2]:
from constants import *

In [3]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [4]:
load_dotenv()

True

In [5]:
mnemonic = os.getenv('mnemonic')

In [6]:
print (mnemonic)

cannon rifle wood empower fish sing address actual report hamster master sort glance vacuum build


In [7]:
def derive_wallets (mnemonic, coin, numderive):

    command = 'php derive -g --mnemonic="'+str(mnemonic)+'" --numderive='+str(numderive)+' --coin='+str(coin)+' --format=jsonpretty' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    return json.loads(output)

In [8]:
coins = {'eth':derive_wallets(mnemonic=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic=mnemonic,coin=BTCTEST,numderive=3)}

In [9]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [10]:
eth_privatekey = coins['eth'][0]['privkey']
btc_privatekey = coins['btc-test'][0]['privkey']

print(json.dumps(eth_privatekey, indent=4, sort_keys=True))
print(json.dumps(btc_privatekey, indent=4, sort_keys=True))

"0xc8f2d8128cb88f7cf065a49a0c22cee301f1dc381073de40681bbb381eb4c3de"
"cVseDmPmf1hH1PaPjzWumctENcqr6dQRyh8q6zXi85TJ54V56tzc"


In [11]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cVseDmPmf1hH1PaPjzWumctENcqr6dQRyh8q6zXi85TJ54V56tzc",
            "pubkey": "021009170bf75d4f77bee4727af2922a3552bc5907904c70d63b7e6ee2c0d2ada7",
            "pubkeyhash": "c5d8f5cfa33ff5912ecf157667f86f20f78988d3",
            "xprv": "tprv8k8QYFfJxbzMhWtMiQpwGXVQvkcaGCwsMunYQv67JMLQhn7LXn7RiR5mhhK9sbpgc3RUZMkPFg7aLnVsQNuS8DRnUQKjsxLxV5iN5w4xcow",
            "xpub": "tpubDGpSgfhZ6yg2ayv9c4VXfw9XVn8WRY8mwDPKhS8Qid8oYGN7AAw1tuhdsnW9aNoLweiPJ3p29w7oVMKsF2imiAnhRoeAv4cnzPdnFgfyJRD"
        },
        {
            "address": "mzViQHWxB5gdoVCThWQ6vTxmMDca6BaVu8",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cU22r46zjcZxRCuNSu48BiHiYPS4dXsQTZd2yUhoynW8bZtZ313Z",
            "pubkey": "036a6561dadf561a7abe2a4d39c426a6a21090c9c46df0e6da2fd33119f5fce70f",
            "pubkeyh

In [12]:
def priv_key_to_account (coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [13]:
eth_account = priv_key_to_account(ETH,eth_privatekey)
btc_account = priv_key_to_account(BTCTEST,btc_privatekey)

In [14]:
print(eth_account)
print(btc_account)

<PrivateKeyTestnet: myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR>


In [15]:
def create_tx(coin,account,recipient,amount):
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [16]:
def send_tx (coin, account, recipient, amount):
    if coin =='eth':
        trxns_eth = create_tx(coin,account, recipient, amount)
        sign_trxns_eth = account.sign_transaction(trxns_eth)
        result = w3.eth.sendRawTransaction(sign_trxns_eth.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trxns_btctest= create_tx(coin,account,recipient,amount)
        sign_trxns_btctest = account.sign_transaction(trxns_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trxns_btctest)       
        return sign_trxns_btctest

### BTC TEST

In [17]:
# create BTC transaction
create_tx(BTCTEST,btc_account,"myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR", 0.001)

'{"unspents":[{"amount":1160723,"confirmations":1,"script":"76a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac","txid":"b278e212bc88752673e98147fc63f3b2ab438392fa7e8f8df76aa0c8162195ca","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":100000,"confirmations":1,"script":"76a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac","txid":"b278e212bc88752673e98147fc63f3b2ab438392fa7e8f8df76aa0c8162195ca","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR",100000],["myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR",1122575]]}'

In [18]:
#send BTC transaction
send_tx(BTCTEST,btc_account,'myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR',0.001)

'0100000002ca952116c8a06af78d8f7efa928343abb2f363fc4781e973267588bc12e278b2010000006a47304402200b7fcde38632384824d8b82d0848a9a4462545657beeb7f7a6901a15e59332dd022003acc6d5d1b23faba3ee7e79e822cdb1a2c1aa524e6c72b770480f9e6e33d8d70121021009170bf75d4f77bee4727af2922a3552bc5907904c70d63b7e6ee2c0d2ada7ffffffffca952116c8a06af78d8f7efa928343abb2f363fc4781e973267588bc12e278b2000000006b4830450221009b0ff09220a6f6ac977bf393c9b0775c12cfccb23db406e7c71c436a4308bd7502206762701ecd18dc04ce6f63ade3227449df6d5304ccd8b3afc187593aaae090150121021009170bf75d4f77bee4727af2922a3552bc5907904c70d63b7e6ee2c0d2ada7ffffffff02a0860100000000001976a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac0f211100000000001976a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac00000000'

### ETH TEST

In [19]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [20]:
from web3 import Web3, HTTPProvider

In [21]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0xbc5A3C2eF27F87bf969036a38BDc326307821D85"))

In [22]:
#checking the Block Number
w3.eth.blockNumber

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: /0xbc5A3C2eF27F87bf969036a38BDc326307821D85 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C4FB020208>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))